# Carbon Capture Analysis

In [5]:
# imports

import pandas as pd
import os

In [3]:
# data folder

output_main = 'output_aggregated'

data_folder = os.path.join(os.path.dirname(os.path.realpath("__file__")),
                           '../data', output_main)

### download files

In [8]:
import boto3

s3 = boto3.client('s3')

s3_bucket = 'tori-calculate-wind-power'

if not os.path.isdir(data_folder):
    os.makedirs(data_folder)

def get_s3_keys(prefix=''):
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=s3_bucket, Prefix=prefix):
        for content in page.get('Contents', ()):
            yield content['Key']


files = list(get_s3_keys(output_main))
for key in files:
    data_file = os.path.join(data_folder, os.path.basename(key))
    s3.download_file(s3_bucket, key, data_file)

files


['output_aggregated/Germany.csv']

### read file

In [43]:
all_files = [os.path.join(data_folder, file_name) for file_name in os.listdir(data_folder)]

df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

df

,latitude,longitude,sed,power_output,carbon_footprint,emissions_carbon_capture
0,47.5,5.625,6.555589e+06,6.331048e+09,0.009697,63566.271188
1,47.5,6.250,6.562429e+06,6.339307e+09,0.009684,63549.690767
2,47.5,6.875,6.621312e+06,5.752784e+09,0.010671,70657.233168
3,47.5,7.500,6.644625e+06,4.296574e+09,0.014288,94937.716507
4,47.5,8.125,6.621909e+06,4.121230e+09,0.014896,98638.609920
...,...,...,...,...,...,...
267,55.5,12.500,6.785806e+06,1.048125e+10,0.005857,39744.691115
268,55.5,13.125,6.778674e+06,1.065198e+10,0.005763,39066.560353
269,55.5,13.750,6.810728e+06,1.075247e+10,0.005709,38884.440760
270,55.5,14.375,6.796221e+06,1.090252e+10,0.005631,38267.607173


### filter data

In [61]:
df['emissions_carbon_capture'] = df['emissions_carbon_capture'].apply(lambda x: 0 if x > 1e10 else x)
# df['emissions_carbon_capture'].sort_values().values

df['carbon_footprint'] = df['carbon_footprint'].apply(lambda x: 0 if x > 1e10 else x)
# df['carbon_footprint'].sort_values().values

## Power Output

In [56]:
import plotly.express as px

fig = px.density_mapbox(df, lat='latitude', lon='longitude', z='power_output', radius=15,
                        zoom=3, mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title = "Data")
fig.show()


## SED

In [57]:
import plotly.express as px

fig = px.density_mapbox(df, lat='latitude', lon='longitude', z='sed', radius=15,
                        zoom=3, mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title = "Data")
fig.show()


## Carbon Footprint

In [53]:
import plotly.express as px

fig = px.density_mapbox(df, lat='latitude', lon='longitude', z='carbon_footprint', radius=20,
                        zoom=3, mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title = "Data")
fig.show()


## Emissions Carbon Capture

In [46]:
import plotly.express as px

fig = px.density_mapbox(df, lat='latitude', lon='longitude', z='emissions_carbon_capture', radius=20,
                        zoom=3, mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title = "Data")
fig.show()
